In [1]:
import pandas as pd
import numpy as np
import os
mingw_path = 'C:\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
import xgboost as xg
import datetime
import sklearn.preprocessing as pp

In [9]:
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
train = pd.read_csv('CrossSellTarget\\Train.csv')

C:\Users\Arkajit\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
test = pd.read_csv('CrossSellTarget\\Test.csv')

C:\Users\Arkajit\Anaconda2\envs\TFENV\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,26,28,29,30,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data_dict = pd.read_csv('CrossSellTarget\\data_dict.csv')
data_dict.drop(['Unnamed: 2','Unnamed: 3'],axis=1, inplace=True)
data_dict.index= data_dict['VARIABLE']

In [44]:
def confusion_mat(train,y_train,col,val): 
    m_y=[]
    m_n=[]
    for i in range(len(val)):
        m_y.append(0)
        m_n.append(0)
    
    for i in train.index.values:
        for j in range(len(val)):
            if( train.get_value(i,col) == val[j]) and (y_train.get_value(i) == 'Y'):
                m_y[j]+=1
            elif( train.get_value(i,col) == val[j]) and (y_train.get_value(i) == 'N'):
                m_n[j]+=1
    print '\tyes','\tno','\t%yes','\t%no','\n'
    for i in range(len(val)):
        print val[i],'\t',m_y[i],'\t',m_n[i],'\t',100 * m_y[i]/(m_y[i] + m_n[i]),'%','\t',100 * m_n[i]/(m_y[i] + m_n[i]),'%','\n'

In [37]:
count=0
b_col=[]
c_col=[]
for i in train.columns:
    if len(train[i].astype('category').cat.categories.get_values()) <= 3:
        if train[i].isnull().sum()<50000:
            #count+=1
            b_col.append([i,list(train[i].astype('category').cat.categories.get_values())])
    elif len(train[i].astype('category').cat.categories.get_values()) > 3:
        if train[i].isnull().sum()<50000:
            count+=1
            c_col.append(i)
    
        
count

19

In [38]:
for col in sorted(b_col):
    print col ,data_dict.get_value(col[0],'DESCRIPTIONS'),train[col[0]].isnull().sum()

['ACT_TYPE', ['CSA', 'SA']] ACCOUNT TYPE - SAVING, CURRENT OR SALARY ACCOUNT 0
['CARD_AUTOMOBILE_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON AUTOMOBILE CATEGORY IN MON_01 0
['CARD_ENTMNT_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON ENTERTAINMENT CATEGORY IN MON_01 0
['CARD_HOBBY_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON HOBBY CATEGORY IN MON_01 0
['CARD_HOME_DECOR_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON HOME AND DECOR CATEGORY IN MON_01 0
['CARD_HOTEL_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON HOTEL CATEGORY IN MON_01 0
['CARD_JEWELLERY_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON JEWELLERY CATEGORY IN MON_01 0
['CARD_MEDICAL_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON MEDICAL CATEGORY IN MON_01 0
['CARD_PRSNL_CARE_MON_01', ['N', 'Y']] TOTAL SPEND THROUGH DEBIT AND CREDIT CARD ON PERSONAL CARECATEGORY IN MON_01 0
['CARD_RESTAURAN

In [45]:
confusion_mat(train,y_train,'COC_ELIGIBLE', [0, 1])

	yes 	no 	%yes 	%no 

0 	3817 	264687 	1 % 	98 % 

1 	795 	30701 	2 % 	97 % 



In [52]:
count=0
for col in sorted(c_col):
    if (str(train[col].dtype) == 'float64') or (str(train[col].dtype) == 'float32'):
        #continue
        count+=1
        print col,' , mean:',' , no.of nan:',train[col].isnull().sum()
        print ' ',data_dict.get_value(col,'DESCRIPTIONS')
count

AMB_MON_01  , mean:  , no.of nan: 0
  MONTHLY BALANCE IN MON_01
AMB_MON_02  , mean:  , no.of nan: 992
  MONTHLY BALANCE IN MON_02
AMB_MON_03  , mean:  , no.of nan: 2778
  MONTHLY BALANCE IN MON_03
AMB_MON_04  , mean:  , no.of nan: 5622
  MONTHLY BALANCE IN MON_04
EOP_BAL_MON_01  , mean:  , no.of nan: 0
  END OF PERIOD BALANCE IN MON_01
EOP_MON_02  , mean:  , no.of nan: 989
  END OF PERIOD BALANCE FOR MON_02
EOP_MON_03  , mean:  , no.of nan: 2768
  END OF PERIOD BALANCE FOR MON_03
N_CASA_MAX_BALANCE_MTD  , mean:  , no.of nan: 0
  CASA MAXIMUM BALANCE - MAXIMUM ACCOUNT BALANCE FOR MON1
N_CASA_MIN_BALANCE_MTD  , mean:  , no.of nan: 0
  CASA MINIMUM BALANCE - MINIMUM ACCOUNT BALANCE FOR MON1
SCRUB_EMI  , mean:  , no.of nan: 0
  MONTHLY TOTAL EMI PAID TO OTHER BANK FOR THE ACTIVE LOANS


10

In [53]:
for col in sorted(c_col):
    if (str(train[col].dtype) == 'float64') or (str(train[col].dtype) == 'float32'):
        if(col.startswith('C') or col.startswith('D')):
            print col#,' , mean:',train[col].median(),' , no.of nan:' ,train[col].isnull().sum()
            print data_dict.get_value(col,'DESCRIPTIONS')

In [21]:
def missing_values_table(df): 
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum()/len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        return mis_val_table_ren_columns 

In [23]:
mis_val_table_ren_columns = missing_values_table(train)

In [24]:
nan_col = list(mis_val_table_ren_columns[mis_val_table_ren_columns['% of Total Values']>= 80].index)

In [26]:
len(nan_col)

234

In [27]:
train.drop(nan_col,axis=1 , inplace=True)
test.drop(nan_col,axis=1 , inplace=True)

In [78]:
y_train = train['RESPONDERS']
train.drop(['RESPONDERS'],axis=1,inplace=True)

In [32]:
zip_tr= train['ZIP_CODE_FINAL']
zip_ts= test['ZIP_CODE_FINAL']
train.drop(['ZIP_CODE_FINAL'] , inplace = True , axis = 1)
test.drop(['ZIP_CODE_FINAL'] , inplace = True , axis = 1)

In [70]:
ind=[]
for ix,row in train.iterrows():
    if row.isnull().sum() > 50 :
        ind.append(ix)

In [72]:
max(ind) 

299426

In [65]:
train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

In [73]:
train.drop(train.index[ind],axis=0,inplace=True)
y_train.drop(y_train.index[ind],axis=0,inplace=True)

In [25]:
for col in train.columns.values:
    if(len(train[col].unique()) == 1):
        nan_col.append(col)

In [29]:
train.dropna(axis=1,how='all' ,inplace=True)
test.dropna(axis=1,how='all',inplace=True)

In [79]:
for i in train.columns:
    if len(train[i].astype('category').cat.categories.get_values()) <= 3:
        train[i] = train[i].astype('category').cat.codes
        test[i] = test[i].astype('category').cat.codes

In [76]:
for i in train.columns:
    if (len(train[i].astype('category').cat.categories.get_values()) > 3) & (str(train[i].dtype) == 'object'):
        #print train[i].isnull().sum(), i 
        train[i] = train[i].astype('category').cat.codes
        test[i] = test[i].astype('category').cat.codes

In [20]:
cat_cols=['LEGAL_ENTITY','DESIGNATION_FINAL','PA_PQ_TAG','NEFT_CC_CATEGORY',
          'NEFT_DC_CATEGORY','TPT_DC_CATEGORY_MON_01','TPT_CC_CATEGORY_MON_01','IMPS_CC_CATEGORY_MON_01']

In [25]:
for col in cat_cols:
    print (col)
    train[col] = train[col].apply(lambda x: x if str(x)  != str(pd.np.nan) else '-1' )
    test[col] = test[col].apply(lambda x: x if str(x)  != str(pd.np.nan) else '-1' )
    train[col] = train[col].astype('str')
    test[col] = test[col].astype('str')
    data= test[col].append(train[col])
    data = data.astype('str')
    le = pp.LabelEncoder().fit(data)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    

LEGAL_ENTITY
DESIGNATION_FINAL
PA_PQ_TAG
NEFT_CC_CATEGORY
NEFT_DC_CATEGORY
TPT_DC_CATEGORY_MON_01
TPT_CC_CATEGORY_MON_01
IMPS_CC_CATEGORY_MON_01


In [26]:
le = pp.LabelEncoder().fit(y_train)
y_train = le.transform(y_train)
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [81]:
for col in train.columns.values:
    train[col] = train[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)
    test[col] = test[col].apply(lambda x: x if str(x) != str(pd.np.nan) else -1)
    #print (col)

In [82]:
train.index = train['CUSTOMER_ID']
train.drop(['CUSTOMER_ID'],axis=1,inplace=True)

In [83]:
test.index = test['CUSTOMER_ID']
test.drop(['CUSTOMER_ID'],axis=1,inplace=True)

In [89]:
cor = train.corr()

In [90]:
cols=[]
i=0
for col in cor.columns.values:
    for index in cor[col].index:
        if(cor.get_value(index,col) > 0.80 or cor.get_value(index,col) < -0.80) and (index != col ) and sorted((index,col)) not in cols:
            i+=1
            tpl = (index,col)
            cols.append(sorted(tpl))
            print ( i,sorted(tpl),'=> ',cor.get_value(index,col))
            

(1, ['D_AMT_L3_MON_06', 'N_CASA_MAX_BALANCE_MTD'], '=> ', 0.83314137462414684)
(2, ['C_AMT_L3_MON_06', 'N_CASA_MAX_BALANCE_MTD'], '=> ', 0.83425485359091389)
(3, ['MAX_C_AMT_L3_MON_06', 'N_CASA_MAX_BALANCE_MTD'], '=> ', 0.83788192479180201)
(4, ['D_AMT_L3_MON_01', 'N_CASA_MAX_BALANCE_MTD'], '=> ', 0.81430503884884742)
(5, ['C_AMT_L3_MON_01', 'N_CASA_MAX_BALANCE_MTD'], '=> ', 0.83371098989266712)
(6, ['MAX_C_AMT_L3_MON_01', 'N_CASA_MAX_BALANCE_MTD'], '=> ', 0.90039793340523677)
(7, ['CC_HOLD_MON_02', 'CC_HOLD_MON_03'], '=> ', 0.96362615053033573)
(8, ['CC_HOLD_MON_01', 'CC_HOLD_MON_03'], '=> ', 0.92163818236690787)
(9, ['CC_HOLD', 'CC_HOLD_MON_03'], '=> ', 0.95819185038176835)
(10, ['CC_HOLD_MON_01', 'CC_HOLD_MON_02'], '=> ', 0.95710208721612922)
(11, ['CC_HOLD', 'CC_HOLD_MON_02'], '=> ', 0.96162821241166807)
(12, ['CC_HOLD', 'CC_HOLD_MON_01'], '=> ', 0.96566490439999564)
(13, ['CC_ACTIVE', 'CC_HOLD_MON_01'], '=> ', 0.81041442650381323)
(14, ['STMT_MON_02', 'STMT_MON_03'], '=> ', 0.8345

In [91]:
cols_0= [i[0] for i in cols]
cols_1= [i[1] for i in cols]
#cols_0 = list(set(cols_0))
cols_0.sort()
#cols_1 = list(set(cols_1))
cols_1.sort()

In [92]:
col_rem=[]
for col in cols_0:
    if col in cols_1:
        col_rem.append(col)
col_rem = list(set(col_rem))

cols_0 = list(set(cols_0))
for col in col_rem:
    if col in cols_0:
        cols_0.remove(col)
len(col_rem)

15

In [93]:
train.drop(col_rem, axis=1,inplace=True)
test.drop(col_rem,axis=1,inplace=True)

In [94]:
len(cols_0)

12

In [95]:
train.drop(cols_0, axis=1,inplace=True)
test.drop(cols_0,axis=1,inplace=True)

In [96]:
train.shape

(233876, 76)

In [97]:
test.shape

(200000, 76)

In [98]:
import shutil
train.to_csv('CrossSellTarget\\PrunedDS\\train_pruned.csv')
test.to_csv('CrossSellTarget\\PrunedDS\\test_pruned.csv')
shutil.make_archive("CrossSellTarget\\PrunedDS", "zip", "CrossSellTarget\\PrunedDS")

'C:\\Users\\Arkajit\\Anaconda2\\envs\\TFENV\\CrossSellTarget\\PrunedDS.zip'

In [143]:
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [46]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from collections import Counter


X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
...                            n_redundant=0, n_repeated=0, n_classes=3,
...                            n_clusters_per_class=1,
...                            weights=[0.01, 0.05, 0.94],
...                            class_sep=0.8, random_state=0)


ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X, y)
print(sorted(Counter(y_resampled).items()))

In [106]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

C:\Users\Arkajit\Anaconda2\envs\TFENV\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Arkajit\Anaconda2\envs\TFENV\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [102]:
x_train = train.as_matrix()
y_train = y_train.as_matrix()
x_test = test.as_matrix()

In [104]:
ros = RandomOverSampler(random_state=0)
print(sorted(Counter(y_train).items()))
X_resampled, y_resampled = ros.fit_sample(x_train, y_train)
print(sorted(Counter(y_resampled).items()))

[(0, 229414), (1, 4462)]
[(0, 229414), (1, 229414)]


In [130]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)

In [134]:
from sklearn.cross_validation import cross_val_score , LeaveOneOut, KFold
from scipy.stats import sem

In [139]:
def cross_val(X_train,y_train,model,K):
    #loo=LeaveOneOut(X_train[:].shape[0])
    cv = KFold(len(y_train),K,shuffle=True,random_state=0)
    scores = cross_val_score(model,X_train,y_train,cv=cv)
    print scores
    print "Mean score: {0:3f} (+/- {1:.3f})".format(np.mean(scores),sem(scores))

In [122]:
#pca = PCA(n_components=20)
#pca.fit(X_resampled)

In [286]:
#score = pca.explained_variance_ratio_
#V = pca.components_

In [124]:
x_train_trans = pca.transform(X_resampled)
x_test_trans = pca.transform(x_test)

In [146]:
#model = LogisticRegression()
#model = SGDClassifier()
#model = LogisticRegressionCV()
model = RandomForestClassifier(max_features = None , n_jobs=2)
#model = XGBClassifier(n_jobs=2)
#model = MultinomialNB()

In [135]:
#model.fit(X_resampled,y_resampled)
model.fit(X_train,y_train)
#model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=2, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [126]:
#y_pred=model.predict(x_test_trans)
y_pred=model.predict(x_test_trans)

In [140]:
cross_val(X_train,y_train,model,5) # xgboost

[ 0.78307657  0.78235462  0.78530758  0.78431319  0.78277393]
Mean score: 0.783565 (+/- 0.001)


In [142]:
cross_val(X_train,y_train,model,5) # LogisticRegressionCV

[ 0.6770463   0.67384523  0.68176865  0.6799842   0.67122541]
Mean score: 0.676774 (+/- 0.002)


In [ ]:
cross_val(X_train,y_train,model,5) # RandomForest

In [127]:
#y_pred = le.inverse_transform(y_pred)

In [128]:
y_pred = pd.DataFrame(y_pred,columns=['RESPONDERS'])
y_pred.index = test.index

In [129]:
import shutil
y_pred.to_csv('CrossSellTarget\\submission\\sample_submission.csv')
shutil.make_archive("CrossSellTarget\\submission", "zip", "CrossSellTarget\\submission")

'C:\\Users\\Arkajit\\Anaconda2\\envs\\TFENV\\CrossSellTarget\\submission.zip'